In [1]:
import numpy as np

#HMM parameters
states = ["Sunny", "Rainy"]
observations = ["Walk", "Shop", "Clean"]

transition_probs = np.array([
    [0.8, 0.2],  # from Sunny -> Sunny / Rainy
    [0.4, 0.6]   # from Rainy -> Sunny / Rainy
])

emission_probs = np.array([
    [0.6, 0.3, 0.1],  # if Sunny: Walk / Shop / Clean
    [0.1, 0.4, 0.5]   # if Rainy: Walk / Shop / Clean
])

initial_probs = np.array([0.7, 0.3])

observed_sequence = ["Walk", "Shop", "Clean"]

obs_to_idx = {obs: idx for idx, obs in enumerate(observations)}


In [ ]:
def viterbi_algorithm(observed_seq, states, initial_probs, transition_probs, emission_probs):
    num_states = len(states)
    num_obs = len(observed_seq)

    # Probability table
    viterbi_probs = np.zeros((num_obs, num_states))
    
    # Backpointer table (to reconstruct the best path)
    backpointer = np.zeros((num_obs, num_states), dtype=int)

    # --- Initialization ---
    for s in range(num_states):
        viterbi_probs[0, s] = initial_probs[s] * emission_probs[s, obs_to_idx[observed_seq[0]]]
        backpointer[0, s] = 0

    # --- Recursion ---
    for t in range(1, num_obs):
        for s in range(num_states):
            # For each current state s, find the previous state that maximizes the probability
            prob_transition = [
                viterbi_probs[t-1, prev_s] * transition_probs[prev_s, s]
                for prev_s in range(num_states)
            ]
            best_prev_state = np.argmax(prob_transition)
            viterbi_probs[t, s] = prob_transition[best_prev_state] * emission_probs[s, obs_to_idx[observed_seq[t]]]
            backpointer[t, s] = best_prev_state

    # --- Termination ---
    best_last_state = np.argmax(viterbi_probs[-1, :])
    best_path_prob = viterbi_probs[-1, best_last_state]

    # --- Path backtracking ---
    best_path = [best_last_state]
    for t in range(num_obs - 1, 0, -1):
        best_path.insert(0, backpointer[t, best_path[0]])

    best_state_sequence = [states[i] for i in best_path]

    return viterbi_probs, backpointer, best_state_sequence, best_path_prob


In [3]:
viterbi_probs, backpointer, best_path, best_prob = viterbi_algorithm(
    observed_sequence, states, initial_probs, transition_probs, emission_probs
)

print("Viterbi probability matrix:")
print(viterbi_probs)


print("\nMost likely hidden state sequence:")
print(best_path)

print("\nProbability of this sequence:")
print(best_prob)


Viterbi probability matrix:
[[0.42     0.03    ]
 [0.1008   0.0336  ]
 [0.008064 0.01008 ]]

Most likely hidden state sequence:
['Sunny', 'Sunny', 'Rainy']

Probability of this sequence:
0.01008
